In [1]:
targetFolderHBK = r"H:\Extracted_Features\HBK\HBK_14285Hz_original_all_features\features"
targetFolderMCC5 = r"H:\Extracted_Features\MCC5\MCC5_12800Hz_original_all_features_motor_vibration_x\features"
targetFolderSIZA = r"H:\Extracted_Features\SIZA\SIZA_original_all_features\features"
normalization_method = "min_max"

# Environment Setup & Imports

In [2]:
import os
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
import numpy as np
import mlflow
from pytorch_lightning.loggers import MLFlowLogger
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, hamming_loss, hinge_loss, jaccard_score, log_loss, precision_score, recall_score, f1_score, make_scorer
from pathlib import Path
from pycaret.classification import * 
from torch import tensor
from torchmetrics.classification import BinaryAccuracy, MulticlassAccuracy
import optuna
import torch
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import GANDALFConfig, CategoryEmbeddingModel,GatedAdditiveTreeEnsembleConfig, NodeConfig, FTTransformerConfig, TabNetModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    ModelConfig,
    TrainerConfig,
    ExperimentConfig,
)
from collections import Counter
from data_loader import load_feature_data

## Helper Functions

In [3]:
def normalizeDataframe(dataframe, normalization_method):
    """
    Normalizes the features of a dataframe using a specified method.

    Args:
        dataframe (pd.DataFrame): The input dataframe with a 'Label' column.
        normalization_method (str): The method to use ("min_max", "z_score", "robust_scaling").

    Returns:
        pd.DataFrame: The dataframe with scaled features.
    """
    # Separate features (X) and the target variable (y)
    y = dataframe['Label']
    X = dataframe.drop(columns=['Label'])

    # Select the scaler based on the chosen method
    if normalization_method == "min_max":
        scaler = MinMaxScaler()
    elif normalization_method == "z_score":
        scaler = StandardScaler()
    elif normalization_method == "robust_scaling":
        scaler = RobustScaler()
    else:
        # Raise an error for an invalid method name
        raise ValueError(f"Unknown normalization_method: '{normalization_method}'")

    # Fit the scaler to the data and transform it
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns,
        index=X.index
    )

    # Rejoin the scaled features with the label column
    df_scaled = X_scaled.join(y)
    
    return df_scaled

In [4]:
def plotPredictionHistograms(df, domain, normalization):
    # 1) mark correct vs incorrect
    df = df.copy()
    df['prediction_quality'] = np.where(
        df['Label'] == df['prediction_label'],
        'correct',
        'incorrect'
    )
    
    # 2) choose a palette (you can override these colors if you like)
    pal = dict(zip(
        ['correct','incorrect'],
        sns.color_palette(n_colors=2)
    ))
    
    skip = {'Label','prediction_label','prediction_score','prediction_quality'}
    for col in df.columns:
        if col in skip:
            continue
        
        fig, ax = plt.subplots(figsize=(8,4))
        sns.histplot(
            data=df, x=col, hue='prediction_quality',
            palette=pal,
            kde=True, multiple='layer', element='step',
            alpha=0.5,
            ax=ax
        )
        
        # 3) build a manual legend using the same palette
        handles = [
            mpatches.Patch(color=pal[k], label=k)
            for k in ['correct','incorrect']
        ]
        ax.legend(
            handles=handles,
            title='Prediction Quality'
        )
        
        ax.set_title(
            f"Distribution of {col} in the '{domain}' domain\n"
            f"(normalization = '{normalization}')"
        )
        ax.set_xlabel(col)
        ax.set_ylabel("Count")
        plt.tight_layout()
        plt.show()

In [5]:
def get_incorrect_predictions(df):
    return df[
        ((df['Label'] == 'damaged')   & (df['prediction_label'] == 'healthy'))
      | ((df['Label'] == 'healthy')  & (df['prediction_label'] == 'damaged'))
    ].copy()

In [6]:
def get_feature_importance_df(model, df):
    importance = model.feature_importances_
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': importance
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)

In [7]:
def get_svm_feature_importance_df(model, df):
    if not hasattr(model, 'coef_'):
        raise ValueError("This SVM model has no coefficients. Use a linear kernel.")
    
    importance = model.coef_.ravel()  # Flatten in case of binary classification
    n = len(importance)
    features = df.columns[:n]
    fi_df = pd.DataFrame({
        'Features': features,
        'importance': abs(importance)
    })
    return fi_df.sort_values(by='importance', ascending=False).reset_index(drop=True)


In [8]:
feature_counter = Counter()
def add_top_features(feature_df: pd.DataFrame, top_n: int):
    top_features = feature_df.nlargest(top_n, 'importance')['Features']
    feature_counter.update(top_features)
    
def plot_feature_importance():
    feature_freq = pd.DataFrame(feature_counter.items(), columns=['Feature', 'Count'])
    plt.figure(figsize=(10, 5))
    sns.barplot(data=feature_freq.sort_values(by='Count', ascending=False),
                x='Feature', y='Count')
    plt.xticks(rotation=45)
    plt.title('Feature Frequency Across Experiments')
    plt.tight_layout()
    plt.show()

## Import Dataset

In [9]:
experiment_name = "Best_Hyperparameters_z_score"

In [10]:
df_binary_HBK = load_feature_data(
    features_path=targetFolderHBK,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 161 files into a DataFrame with shape (200093, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (200093, 28)


In [11]:
df_binary_SIZA = load_feature_data(
    features_path=targetFolderSIZA,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [12]:
df_binary_MCC5 = load_feature_data(
    features_path=targetFolderMCC5,
    include_augmentations=False,      # Only 'original' data
    include_speed_torque=False,       # Drop operating conditions
    binary_classification=True,       # 'healthy' vs 'damaged'
)

Successfully loaded 36 files into a DataFrame with shape (53928, 30)
Applied binary classification: 'healthy' vs 'damaged'.
Dropped 'Speed' and 'Torque' columns.
Final DataFrame shape: (53928, 28)


In [13]:
combined_df = pd.concat([df_binary_HBK, df_binary_SIZA, df_binary_MCC5], ignore_index=True)

In [14]:
normalized_df = normalizeDataframe(combined_df, normalization_method)

In [15]:
features_df_training_normalized, features_df_testing_normalized = train_test_split(
    normalized_df, 
    test_size=0.2,    # e.g., 20% for testing
    random_state=42   # for reproducibility
)

# Experiment Setup (ML)

## Setup Hyperparameters

In [16]:
experiment = setup(features_df_training_normalized, target='Label', log_experiment = True, experiment_name = experiment_name, use_gpu = True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Quadro P620, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split 

,Description,Value
0,Session id,5991
1,Target,Label
2,Target type,Binary
3,Target mapping,"damaged: 0, healthy: 1"
4,Original data shape,"(246359, 28)"
5,Transformed data shape,"(246359, 28)"
6,Transformed train set shape,"(172451, 28)"
7,Transformed test set shape,"(73908, 28)"
8,Numeric features,27
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Quadro P620, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split 

## Add aditional metrics

In [17]:
# Binary classification metrics
add_metric('balanced_acc', 'Balance Acc', balanced_accuracy_score, target='pred', greater_is_better=True)
add_metric('hamming_loss', 'Hamming Loss', hamming_loss, target='pred', greater_is_better=False)
add_metric('jaccard_score', 'Jaccard Score', jaccard_score, target='pred', greater_is_better=True)
add_metric('log_loss', 'Log Loss', log_loss, target='pred_proba', greater_is_better=False)

Name                                                          Log Loss
Display Name                                                  Log Loss
Score Function       <pycaret.internal.metrics.EncodedDecodedLabels...
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: log_loss, dtype: object

In [18]:
all_metrics = get_metrics()
all_metrics

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x0000023A434F2B60>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x0000023A434F2...,"make_scorer(cohen_kappa_score, response_method...",pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x0000023A434F3...,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False
balanced_acc,Balance Acc,Balance Acc,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(balanced_accuracy_score, response_...",pred,{},True,True,True
hamming_loss,Hamming Loss,Hamming Loss,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(hamming_loss, greater_is_better=Fa...",pred,{},False,True,True


In [19]:
num_iterations_tuning = 10
optimized_metric = 'F1'

## Light Gradient Boosting Machine

In [20]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9534,0.9873,0.9534,0.9543,0.9518,0.8582,0.8637,0.9042,0.0466,0.7975,0.1122
1,0.9525,0.9865,0.9525,0.9537,0.9508,0.8550,0.8612,0.9011,0.0475,0.7930,0.1140
2,0.9490,0.9867,0.9490,0.9502,0.9471,0.8439,0.8506,0.8947,0.0510,0.7786,0.1166
3,0.9489,0.9852,0.9489,0.9498,0.9471,0.8441,0.8501,0.8961,0.0511,0.7791,0.1199
4,0.9537,0.9868,0.9537,0.9548,0.9521,0.8588,0.8646,0.9037,0.0463,0.7981,0.1136
5,0.9533,0.9871,0.9533,0.9541,0.9519,0.8584,0.8634,0.9054,0.0467,0.7979,0.1121
6,0.9534,0.9867,0.9534,0.9546,0.9518,0.8578,0.8638,0.9028,0.0466,0.7967,0.1140
7,0.9527,0.9871,0.9527,0.9538,0.9511,0.8558,0.8617,0.9021,0.0473,0.7941,0.1136
8,0.9507,0.9860,0.9507,0.9515,0.9490,0.8497,0.8554,0.8994,0.0493,0.7864,0.1171


2025/09/26 16:59:36 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [21]:
lightgbm_tuned_model, lightgbm_tuner = tune_model(lightgbm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9627,0.9904,0.9627,0.9623,0.9622,0.8906,0.8914,0.9353,0.0373,0.8424,0.0958
1,0.9627,0.9900,0.9627,0.9623,0.9622,0.8906,0.8914,0.9355,0.0373,0.8424,0.0984
2,0.9632,0.9898,0.9632,0.9629,0.9627,0.8919,0.8928,0.9353,0.0368,0.8441,0.1091
3,0.9621,0.9900,0.9621,0.9618,0.9617,0.8892,0.8899,0.9351,0.0379,0.8405,0.1065
4,0.9638,0.9894,0.9638,0.9635,0.9634,0.8940,0.8946,0.9378,0.0362,0.8470,0.1068
5,0.9654,0.9907,0.9654,0.9652,0.9651,0.8990,0.8996,0.9406,0.0346,0.8537,0.0945
6,0.9646,0.9913,0.9646,0.9643,0.9642,0.8962,0.8970,0.9383,0.0354,0.8499,0.0960
7,0.9622,0.9885,0.9622,0.9619,0.9619,0.8898,0.8903,0.9366,0.0378,0.8416,0.1745
8,0.9617,0.9901,0.9617,0.9613,0.9613,0.8880,0.8886,0.9354,0.0383,0.8392,0.1077


[LightGBM] [Warning] feature_fraction is set=0.577751533396215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.577751533396215
[LightGBM] [Warning] bagging_fraction is set=0.9469401484093165, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9469401484093165
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.577751533396215, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.577751533396215
[LightGBM] [Warning] bagging_fraction is set=0.9469401484093165, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9469401484093165
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Info] Number of positive: 34970, number of negative: 120235
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6885
[LightGBM] [Info] Number of data poi

2025/09/26 17:16:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [22]:
print(lightgbm_tuned_model)

LGBMClassifier(bagging_fraction=0.6852666247712094, bagging_freq=6,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device='gpu', feature_fraction=0.8474520015916083,
               importance_type='split', learning_rate=0.44396855094714766,
               max_depth=-1, min_child_samples=18, min_child_weight=0.001,
               min_split_gain=0.9165929677584167, n_estimators=46, n_jobs=-1,
               num_leaves=126, objective=None, random_state=5991,
               reg_alpha=3.497504838524017e-10,
               reg_lambda=1.0536166171914501e-05, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


In [23]:
evaluate_model(lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [24]:
lightgbm_top_features = get_feature_importance_df(lightgbm, features_df_training_normalized)
lightgbm_top_features

,Features,importance
0,meanFreq,325
1,bandwidth,312
2,meanWavelet,287
3,medianFreq,281
4,spectral_flatness,278
5,skewness,182
6,entropyWavelet,167
7,spectral_skewness,162
8,spectral_kurtosis,112
9,varWavelet,109


In [55]:
predictions_lightgbm = predict_model(lightgbm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9641,0.9891,0.9641,0.9638,0.9636,0.8948,0.8955,0.9378,0.0359,0.8480,0.1351


[LightGBM] [Warning] feature_fraction is set=0.8474520015916083, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8474520015916083
[LightGBM] [Warning] bagging_fraction is set=0.6852666247712094, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6852666247712094
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8474520015916083, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8474520015916083
[LightGBM] [Warning] bagging_fraction is set=0.6852666247712094, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6852666247712094
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


In [25]:
predictions_lightgbm = predict_model(lightgbm, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Light Gradient Boosting Machine,0.9521,0.9867,0.9521,0.9530,0.9504,0.8541,0.8597,0.9018,0.0479,0.7920,0.1141


In [26]:
predictions_lightgbm

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
93960,0.552991,0.014920,0.016479,0.425787,0.002338,0.032911,0.065007,0.016130,0.027697,0.052295,...,0.000159,0.990514,0.000349,0.000205,2.437040e-08,3.937714e-07,2.481956e-08,damaged,damaged,0.9990
371,0.552591,0.003928,0.004507,0.423880,0.002797,0.010123,0.079416,0.019870,0.016115,0.057731,...,0.000032,0.990030,0.000038,0.000016,5.488772e-11,1.345334e-09,6.012260e-11,healthy,healthy,0.9804
33266,0.552529,0.003798,0.004368,0.423978,0.002739,0.011199,0.088883,0.021954,0.017229,0.056826,...,0.000035,0.990020,0.000035,0.000016,1.947031e-10,3.991882e-09,1.963406e-10,healthy,healthy,0.9978
238774,0.551901,0.055206,0.060463,0.426438,0.002858,0.151513,0.102395,0.025197,0.076742,0.058213,...,0.006693,0.993469,0.007301,0.003240,2.727015e-06,3.764719e-05,2.924497e-06,damaged,damaged,0.9298
210317,0.551909,0.021319,0.023460,0.425487,0.003348,0.052560,0.076691,0.020320,0.039388,0.069841,...,0.001073,0.990896,0.001099,0.000473,1.558190e-07,2.279038e-06,1.575380e-07,damaged,damaged,0.9209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241132,0.551992,0.022300,0.024539,0.424149,0.005463,0.073776,0.113589,0.031148,0.056360,0.086858,...,0.001233,0.990859,0.001203,0.000570,3.148032e-07,4.255994e-06,3.150592e-07,damaged,damaged,0.7937
214791,0.551964,0.024686,0.027143,0.424141,0.005454,0.082537,0.110840,0.029353,0.056729,0.077346,...,0.001427,0.991052,0.001472,0.000726,4.999926e-07,6.492028e-06,5.006992e-07,damaged,damaged,0.8331
165165,0.552578,0.092093,0.100734,0.411164,0.008022,0.384982,0.169760,0.044862,0.158203,0.083977,...,0.017095,0.992674,0.017984,0.007493,5.570935e-06,8.848780e-05,6.855859e-06,damaged,damaged,0.9999
217690,0.552060,0.089336,0.097720,0.429396,0.002707,0.214175,0.078653,0.019528,0.080315,0.055886,...,0.017375,0.993681,0.019038,0.008986,3.116306e-05,3.357050e-04,3.242283e-05,damaged,damaged,0.7248


In [27]:
get_incorrect_predictions(predictions_lightgbm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
258524,0.551992,0.002439,0.002697,0.420304,0.004412,0.007740,0.113671,0.029747,0.017695,0.074810,...,0.000011,0.989986,0.000014,0.000008,4.953652e-11,1.070947e-09,5.001858e-11,healthy,damaged,0.6944
207745,0.551983,0.064504,0.070614,0.422450,0.001964,0.129561,0.058001,0.014165,0.054014,0.047083,...,0.009980,0.992818,0.009960,0.004548,1.246231e-05,1.371142e-04,1.267123e-05,healthy,damaged,0.8420
255723,0.551969,0.049521,0.054260,0.426228,0.002760,0.114917,0.071149,0.018054,0.055952,0.058699,...,0.005488,0.992932,0.005880,0.002727,2.272309e-06,3.087725e-05,2.403533e-06,healthy,damaged,0.7625
207040,0.552078,0.061361,0.067186,0.426386,0.003138,0.172687,0.088008,0.022221,0.073193,0.061610,...,0.008198,0.993852,0.009018,0.004295,5.222147e-06,6.780543e-05,5.558231e-06,healthy,damaged,0.6417
254381,0.552025,0.025862,0.028432,0.423788,0.004653,0.072528,0.085672,0.023867,0.049086,0.084438,...,0.001710,0.991014,0.001616,0.000722,6.144647e-07,7.738488e-06,6.121063e-07,healthy,damaged,0.8711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200497,0.551902,0.026917,0.029575,0.425166,0.003908,0.075176,0.090681,0.024281,0.051047,0.076014,...,0.001828,0.991111,0.001748,0.000831,5.012502e-07,6.812054e-06,5.060598e-07,healthy,damaged,0.5772
260610,0.552007,0.057093,0.062526,0.424157,0.003021,0.146886,0.080000,0.020677,0.066490,0.065090,...,0.007284,0.993342,0.007811,0.003527,4.189857e-06,5.363786e-05,4.399530e-06,healthy,damaged,0.5103
258687,0.552024,0.023971,0.026367,0.425718,0.003892,0.063684,0.079928,0.021201,0.043132,0.071010,...,0.001458,0.990960,0.001389,0.000688,3.881280e-07,5.331666e-06,3.902922e-07,healthy,damaged,0.6065
205706,0.551965,0.022341,0.024582,0.425170,0.003709,0.063984,0.088835,0.022831,0.044268,0.065854,...,0.001195,0.990959,0.001207,0.000591,2.401605e-07,3.405097e-06,2.429444e-07,healthy,damaged,0.6497


## Random Forest Classifier

In [28]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9769,0.9954,0.9769,0.9771,0.9766,0.9320,0.9333,0.9531,0.0231,0.8988,0.0757
1,0.9759,0.9952,0.9759,0.9761,0.9755,0.9289,0.9302,0.9512,0.0241,0.8943,0.0762
2,0.9754,0.9956,0.9754,0.9756,0.9750,0.9273,0.9287,0.9503,0.0246,0.8921,0.0759
3,0.9755,0.9951,0.9755,0.9758,0.9751,0.9278,0.9292,0.9502,0.0245,0.8927,0.0763
4,0.9768,0.9950,0.9768,0.9769,0.9765,0.9318,0.9329,0.9540,0.0232,0.8986,0.0758
5,0.9757,0.9949,0.9757,0.9758,0.9754,0.9285,0.9297,0.9520,0.0243,0.8940,0.0774
6,0.9777,0.9963,0.9777,0.9780,0.9774,0.9344,0.9356,0.9545,0.0223,0.9023,0.0728
7,0.9767,0.9956,0.9767,0.9770,0.9764,0.9314,0.9328,0.9522,0.0233,0.8979,0.0762
8,0.9741,0.9950,0.9741,0.9743,0.9737,0.9238,0.9251,0.9488,0.0259,0.8872,0.0782


2025/09/26 17:19:49 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [29]:
rf_tuned_model, rf_tuner = tune_model(rf, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.9295,0.9826,0.9295,0.9327,0.9306,0.8045,0.8058,0.9154,0.0705,0.7398,0.1635
1,0.9296,0.9810,0.9296,0.9322,0.9305,0.8039,0.8049,0.9133,0.0704,0.7388,0.1665
2,0.9295,0.9821,0.9295,0.9328,0.9306,0.8047,0.8061,0.9158,0.0705,0.7402,0.1645
3,0.9242,0.9802,0.9242,0.9280,0.9254,0.7904,0.7921,0.9098,0.0758,0.7241,0.1700
4,0.9306,0.9817,0.9306,0.9330,0.9315,0.8063,0.8071,0.9135,0.0694,0.7413,0.1645
5,0.9270,0.9824,0.9270,0.9314,0.9284,0.7992,0.8013,0.9162,0.0730,0.7346,0.1640
6,0.9337,0.9829,0.9337,0.9359,0.9345,0.8148,0.8156,0.9179,0.0663,0.7513,0.1638
7,0.9287,0.9823,0.9287,0.9324,0.9299,0.8030,0.8047,0.9161,0.0713,0.7385,0.1636
8,0.9331,0.9822,0.9331,0.9355,0.9339,0.8134,0.8143,0.9178,0.0669,0.7498,0.1662


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


2025/09/26 18:25:54 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [30]:
print(rf_tuned_model)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=5991, verbose=0,
                       warm_start=False)


In [56]:
predictions_rf = predict_model(rf_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9792,0.9964,0.9792,0.9793,0.9789,0.9388,0.9398,0.9579,0.0208,0.9086,0.0693


In [31]:
evaluate_model(rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [32]:
rf_top_features = get_feature_importance_df(rf, features_df_training_normalized)
rf_top_features

,Features,importance
0,meanWavelet,0.103067
1,medianFreq,0.080374
2,std,0.064350
3,meanFreq,0.058213
4,spectral_entropy,0.056627
5,energyWavelet,0.053412
6,bandwidth,0.051480
7,spectral_flatness,0.051151
8,energy,0.049902
9,rms,0.039340


In [33]:
predictions_rf = predict_model(rf, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,Random Forest Classifier,0.9792,0.9964,0.9792,0.9793,0.9789,0.9388,0.9398,0.9579,0.0208,0.9086,0.0693


In [34]:
get_incorrect_predictions(predictions_rf)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label,prediction_score
255723,0.551969,0.049521,0.054260,0.426228,0.002760,0.114917,0.071149,0.018054,0.055952,0.058699,...,0.005488,0.992932,0.005880,0.002727,2.272309e-06,0.000031,2.403533e-06,healthy,damaged,0.67
207040,0.552078,0.061361,0.067186,0.426386,0.003138,0.172687,0.088008,0.022221,0.073193,0.061610,...,0.008198,0.993852,0.009018,0.004295,5.222147e-06,0.000068,5.558231e-06,healthy,damaged,0.67
254381,0.552025,0.025862,0.028432,0.423788,0.004653,0.072528,0.085672,0.023867,0.049086,0.084438,...,0.001710,0.991014,0.001616,0.000722,6.144647e-07,0.000008,6.121063e-07,healthy,damaged,0.87
202856,0.551827,0.054470,0.059656,0.425756,0.003219,0.140550,0.082431,0.021076,0.065754,0.063237,...,0.006539,0.993346,0.007109,0.003369,3.546623e-06,0.000046,3.744839e-06,healthy,damaged,0.86
212285,0.551888,0.067535,0.073920,0.425232,0.002661,0.176425,0.084050,0.020632,0.072623,0.054543,...,0.009619,0.993919,0.010887,0.005161,8.933521e-06,0.000109,9.383444e-06,damaged,healthy,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240598,0.551993,0.026294,0.028902,0.424818,0.004320,0.071024,0.090784,0.024753,0.051131,0.080478,...,0.001796,0.991025,0.001669,0.000843,6.234846e-07,0.000008,6.257166e-07,damaged,healthy,0.58
202298,0.551931,0.048106,0.052713,0.426111,0.002825,0.130462,0.082664,0.020452,0.060604,0.055979,...,0.005076,0.992947,0.005549,0.002682,2.248988e-06,0.000031,2.374636e-06,healthy,damaged,0.59
206916,0.552133,0.052712,0.057748,0.427125,0.002644,0.126636,0.072806,0.018312,0.058405,0.057251,...,0.006104,0.993283,0.006661,0.003247,3.320081e-06,0.000044,3.503566e-06,healthy,damaged,0.88
200497,0.551902,0.026917,0.029575,0.425166,0.003908,0.075176,0.090681,0.024281,0.051047,0.076014,...,0.001828,0.991111,0.001748,0.000831,5.012502e-07,0.000007,5.060598e-07,healthy,damaged,0.59


## SVM

In [35]:
svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8371,0.8319,0.8371,0.8384,0.8377,0.5369,0.5369,0.7704,0.1629,0.4731,-0.0000
1,0.8502,0.8260,0.8502,0.8414,0.8413,0.5254,0.5354,0.7375,0.1498,0.4446,-0.0000
2,0.8415,0.8229,0.8415,0.8373,0.8390,0.5321,0.5329,0.7583,0.1585,0.4631,-0.0000
3,0.8359,0.8205,0.8359,0.8338,0.8348,0.5235,0.5237,0.7583,0.1641,0.4586,-0.0000
4,0.8430,0.8288,0.8430,0.8406,0.8417,0.5429,0.5431,0.7671,0.1570,0.4744,-0.0000
5,0.8312,0.8241,0.8312,0.8316,0.8314,0.5177,0.5177,0.7595,0.1688,0.4564,-0.0000
6,0.8424,0.8242,0.8424,0.8383,0.8400,0.5352,0.5359,0.7599,0.1576,0.4657,-0.0000
7,0.8327,0.8218,0.8327,0.8324,0.8325,0.5199,0.5199,0.7595,0.1673,0.4575,-0.0000
8,0.8379,0.8253,0.8379,0.8366,0.8372,0.5319,0.5319,0.7637,0.1621,0.4664,-0.0000


2025/09/26 18:26:13 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [36]:
svm_tuned_model, svm_tuner = tune_model(svm, search_library = 'optuna', return_tuner=True, n_iter=num_iterations_tuning, optimize=optimized_metric)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
Fold,,,,,,,,,,,
0,0.8792,0.8751,0.8792,0.8769,0.8694,0.6056,0.6261,0.7657,0.1208,0.5105,-0.0000
1,0.8498,0.8689,0.8498,0.8557,0.8253,0.4595,0.5192,0.6830,0.1502,0.3626,-0.0000
2,0.8542,0.8666,0.8542,0.8570,0.8331,0.4855,0.5353,0.6971,0.1458,0.3885,-0.0000
3,0.8576,0.8652,0.8576,0.8597,0.8383,0.5027,0.5483,0.7061,0.1424,0.4051,-0.0000
4,0.8674,0.8759,0.8674,0.8710,0.8507,0.5418,0.5844,0.7246,0.1326,0.4412,-0.0000
5,0.8757,0.8707,0.8757,0.8709,0.8681,0.6049,0.6174,0.7724,0.1243,0.5145,-0.0000
6,0.8563,0.8685,0.8563,0.8588,0.8363,0.4960,0.5435,0.7025,0.1437,0.3985,-0.0000
7,0.8727,0.8668,0.8727,0.8677,0.8641,0.5918,0.6062,0.7644,0.1273,0.5010,-0.0000
8,0.8755,0.8688,0.8755,0.8702,0.8701,0.6150,0.6216,0.7848,0.1245,0.5286,-0.0000


2025/09/26 18:27:29 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [37]:
print(svm_tuned_model)

SGDClassifier(alpha=1.5824807951899164e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0015152815364764712,
              fit_intercept=False, l1_ratio=0.7894968232784667,
              learning_rate='optimal', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l1', power_t=0.5,
              random_state=5991, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)


In [57]:
predictions_svm = predict_model(svm_tuned_model, data = features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8790,0,0.8790,0.8747,0.8714,0.6150,0.6279,0.7768,0.1210,0.5239,0


In [38]:
evaluate_model(svm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [39]:
svm_top_features = get_svm_feature_importance_df(svm, features_df_training_normalized)
svm_top_features

,Features,importance
0,clearance,8.123575
1,skewness,5.706661
2,spectral_flatness,4.921232
3,spectral_kurtosis,4.793021
4,meanWavelet,4.465211
5,spectral_skewness,4.383496
6,spectral_entropy,3.915387
7,medianFreq,3.805859
8,ptp,3.391994
9,rms,2.846113


In [40]:
predictions_svm = predict_model(svm, data=features_df_testing_normalized)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Balance Acc,Hamming Loss,Jaccard Score,Log Loss
0,SVM - Linear Kernel,0.8547,0,0.8547,0.8468,0.8469,0.5437,0.5524,0.7478,0.1453,0.4619,0


In [41]:
get_incorrect_predictions(predictions_svm)

,mean,rms,std,skewness,kurtosis,ptp,crest,impulse,clearance,shape,...,meanWavelet,varWavelet,entropyWavelet,energyWavelet,meanSpectrogram,varSpectrogram,entropySpectrogram,energySpectrogram,Label,prediction_label
118771,0.552559,0.000958,0.001264,0.432376,0.002516,0.002636,0.072893,0.018157,0.007135,0.055219,...,0.000994,6.606178e-07,0.989957,9.603716e-07,0.000001,1.281038e-12,3.123863e-11,1.283364e-12,healthy,damaged
261113,0.551990,0.066536,0.072833,0.425662,0.002485,0.161186,0.077418,0.019200,0.068356,0.055364,...,0.084564,9.588812e-03,0.994078,1.059881e-02,0.005050,7.198084e-06,9.151653e-05,7.663102e-06,healthy,damaged
207745,0.551983,0.064504,0.070614,0.422450,0.001964,0.129561,0.058001,0.014165,0.054014,0.047083,...,0.071651,9.979729e-03,0.992818,9.959937e-03,0.004548,1.246231e-05,1.371142e-04,1.267123e-05,healthy,damaged
45208,0.552708,0.014590,0.016141,0.422904,0.002285,0.033954,0.073914,0.017874,0.029609,0.049267,...,0.016197,5.208895e-04,0.990540,5.172486e-04,0.000233,3.886344e-08,6.216078e-07,3.925577e-08,damaged,healthy
255723,0.551969,0.049521,0.054260,0.426228,0.002760,0.114917,0.071149,0.018054,0.055952,0.058699,...,0.060751,5.488139e-03,0.992932,5.879610e-03,0.002727,2.272309e-06,3.087725e-05,2.403533e-06,healthy,damaged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260610,0.552007,0.057093,0.062526,0.424157,0.003021,0.146886,0.080000,0.020677,0.066490,0.065090,...,0.070105,7.284229e-03,0.993342,7.811357e-03,0.003527,4.189857e-06,5.363786e-05,4.399530e-06,healthy,damaged
258687,0.552024,0.023971,0.026367,0.425718,0.003892,0.063684,0.079928,0.021201,0.043132,0.071010,...,0.024665,1.457979e-03,0.990960,1.389366e-03,0.000688,3.881280e-07,5.331666e-06,3.902922e-07,healthy,damaged
83458,0.552707,0.022745,0.025044,0.421023,0.002244,0.047887,0.059128,0.014821,0.032424,0.052496,...,0.028913,1.133101e-03,0.991231,1.248235e-03,0.000533,8.785566e-08,1.485983e-06,9.285375e-08,damaged,healthy
205706,0.551965,0.022341,0.024582,0.425170,0.003709,0.063984,0.088835,0.022831,0.044268,0.065854,...,0.025421,1.195106e-03,0.990959,1.207305e-03,0.000591,2.401605e-07,3.405097e-06,2.429444e-07,healthy,damaged


# Experiment Setup (DL)

## Configure Data

In [42]:
train, test = train_test_split(features_df_training_normalized, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

Train Shape: (157669, 28) | Val Shape: (39418, 28) | Test Shape: (49272, 28)


In [43]:
target = "Label"

categorical_cols = [
    col
    for col in features_df_training_normalized.select_dtypes(include=["object","category"]).columns
    if col != target
]

continuous_cols = features_df_training_normalized.select_dtypes(include=["number"]).columns.tolist()

In [44]:
print("Target:", target)
print("Categorical inputs:", categorical_cols)  
print("Continuous inputs:", continuous_cols)    

Target: Label
Categorical inputs: []
Continuous inputs: ['mean', 'rms', 'std', 'skewness', 'kurtosis', 'ptp', 'crest', 'impulse', 'clearance', 'shape', 'energy', 'entropy', 'meanFreq', 'medianFreq', 'bandwidth', 'spectral_flatness', 'spectral_entropy', 'spectral_skewness', 'spectral_kurtosis', 'meanWavelet', 'varWavelet', 'entropyWavelet', 'energyWavelet', 'meanSpectrogram', 'varSpectrogram', 'entropySpectrogram', 'energySpectrogram']


In [45]:
data_config = DataConfig(
    target=[target],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

In [46]:
available_gpu=1 if torch.cuda.is_available() else 0
print(f"Available GPU: {'Yes' if available_gpu else 'No'}")

Available GPU: Yes


In [47]:
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=20,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    batch_size=256,
)

optimizer_config = OptimizerConfig()

experiment_config = ExperimentConfig(
        project_name="TEST",
        run_name="test",
        log_target="tensorboard",
    )

In [48]:
n_trials = 10

## Tabnet

In [49]:
def TabNet_Optimization(trial):
    n_d     = trial.suggest_int("n_d", 4, 64)
    n_a     = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma   = trial.suggest_float("gamma", 1.0, 2.0)
    embedding_dropout = trial.suggest_float("embedding_dropout", 0, 1)
    lr      = trial.suggest_loguniform("learning_rate", 1e-5, 1e-1)
    
    tabnet_config = TabNetModelConfig(
        task="classification",
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        embedding_dropout=embedding_dropout,
        learning_rate=lr,
        n_independent=2,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )
    
    tabnet_model = TabularModel(
        data_config=data_config,
        model_config=tabnet_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    
    tabnet_model.fit(train=train, validation=val)

    preds_df = tabnet_model.predict(val)
    y_pred = preds_df["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [50]:
tabnet_study = optuna.create_study(direction="maximize")
tabnet_study.optimize(TabNet_Optimization, n_trials=n_trials)

2025-09-26 18:27:33,327 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 18:27:33,379 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 18:27:33,399 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 18:27:33,561 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: TabNetModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _embedding_layer │ Identity         │      0 │ train │
│ 1 │ _backbone        │ TabNetBackbone   │  136 K │ train │
│ 2 │ _head            │ Identity         │      0 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 136 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 136 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 251                                                                                         
Modules in eval mode: 0

Output()

2025-09-26 19:52:27,375 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 19:52:27,375 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [51]:
print("Best params:", tabnet_study.best_params)
print("Best F1 score:", tabnet_study.best_value)

Best params: {'n_d': 57, 'n_a': 43, 'n_steps': 4, 'gamma': 1.128222664472358, 'embedding_dropout': 0.4935754302306584, 'learning_rate': 5.994667027948612e-05}
Best F1 score: 0.9063968516829841


In [58]:
tabnet_study.best_trial

FrozenTrial(number=8, state=1, values=[0.9063968516829841], datetime_start=datetime.datetime(2025, 9, 26, 19, 25, 54, 70482), datetime_complete=datetime.datetime(2025, 9, 26, 19, 41, 57, 607393), params={'n_d': 57, 'n_a': 43, 'n_steps': 4, 'gamma': 1.128222664472358, 'embedding_dropout': 0.4935754302306584, 'learning_rate': 5.994667027948612e-05}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_d': IntDistribution(high=64, log=False, low=4, step=1), 'n_a': IntDistribution(high=64, log=False, low=4, step=1), 'n_steps': IntDistribution(high=10, log=False, low=3, step=1), 'gamma': FloatDistribution(high=2.0, log=False, low=1.0, step=None), 'embedding_dropout': FloatDistribution(high=1.0, log=False, low=0.0, step=None), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None)}, trial_id=8, value=None)

## GANDALF

In [52]:
def GANDALF_Optimization(trial):
    gflu_stages               = trial.suggest_int("gflu_stages", 1, 10)
    gflu_dropout              = trial.suggest_float("gflu_dropout", 0.0, 0.5)
    gflu_feature_init_sparsity = trial.suggest_float("gflu_feature_init_sparsity", 0.1, 0.9)
    learnable_sparsity        = trial.suggest_categorical("learnable_sparsity", [True, False])
    embedding_dropout         = trial.suggest_float("embedding_dropout", 0.0, 0.5)
    batch_norm_continuous     = trial.suggest_categorical("batch_norm_continuous_input", [True, False])
    learning_rate             = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # 2) build Gandalf config
    gandalf_config = GANDALFConfig(
        task="classification",
        gflu_stages=gflu_stages,
        gflu_dropout=gflu_dropout,
        gflu_feature_init_sparsity=gflu_feature_init_sparsity,
        learnable_sparsity=learnable_sparsity,
        embedding_dropout=embedding_dropout,
        batch_norm_continuous_input=batch_norm_continuous,
        learning_rate=learning_rate,
        metrics=[
            "auroc",
            "recall",
            "precision",
            "f1_score",
            "cohen_kappa",
            "matthews_corrcoef",
            "hamming_distance",
            "jaccard_index",
        ],
        metrics_prob_input=[
            True,   # auroc
            False,  # recall
            False,  # precision
            False,  # f1_score
            False,  # cohen_kappa
            False,  # matthews_corrcoef
            False,  # hamming_distance
            False,  # jaccard_index
        ],
        metrics_params=[
            {"average": "macro", "num_classes": 2},  # auroc
            {"average": "macro", "num_classes": 2},  # recall
            {"average": "macro", "num_classes": 2},  # precision
            {"average": "macro", "num_classes": 2},  # f1_score
            {"num_classes": 2},                      # cohen_kappa
            {},                                      # matthews_corrcoef
            {},                                      # hamming_distance
            {"average": "macro", "num_classes": 2},  # jaccard_index
        ]
    )

    # 3) instantiate & train
    model = TabularModel(
        data_config=data_config,
        model_config=gandalf_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        verbose=True
    )
    model.fit(train=train, validation=val)

    # 4) predict & return macro-F1
    preds = model.predict(val)
    y_pred = preds["Label_prediction"].to_numpy()
    y_true = val["Label"].to_numpy()
    return f1_score(y_true, y_pred, average="macro")

In [53]:
gandalf_study = optuna.create_study(direction="maximize")
gandalf_study.optimize(GANDALF_Optimization, n_trials=n_trials)

2025-09-26 19:52:35,170 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-09-26 19:52:35,211 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-09-26 19:52:35,239 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
classification task

2025-09-26 19:52:35,421 - {pytorch_tabular.tabular_model:599} - INFO - Preparing the Model: GANDALFModel

...

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ _backbone        │ GANDALFBackbone  │ 40.3 K │ train │
│ 1 │ _embedding_layer │ Embedding1dLayer │     54 │ train │
│ 2 │ _head            │ Sequential       │     58 │ train │
│ 3 │ loss             │ CrossEntropyLoss │      0 │ train │
└───┴──────────────────┴──────────────────┴────────┴───────┘

Trainable params: 40.5 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 40.5 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 34                                                                                          
Modules in eval mode: 0

Output()

2025-09-26 20:34:44,398 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-09-26 20:34:44,398 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

In [54]:
print("Best params:", gandalf_study.best_params)
print("Best F1 score:", gandalf_study.best_value)

Best params: {'gflu_stages': 6, 'gflu_dropout': 0.18162968859851508, 'gflu_feature_init_sparsity': 0.33034707998094537, 'learnable_sparsity': False, 'embedding_dropout': 0.17899966066777218, 'batch_norm_continuous_input': True, 'learning_rate': 4.572560736461007e-05}
Best F1 score: 0.8986825496869573
